In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import impute

In [5]:
# importo el dataset de properati
properati = pd.read_csv('./../../entregas/entrega1/dataset/datos_properati.csv', delimiter=',', parse_dates = ['created_on'])

In [6]:
# queremos saber que columnas tienen NaN y cuantos
properati.isnull().sum()

created_on                     0
operation                      0
property_type                  0
place_with_parent_names        0
lat-lon                        0
lat                            0
lon                            0
price_aprox_usd             1534
surface_total_in_m2         2393
surface_covered_in_m2       2103
price_usd_per_m2            2942
floor                      16223
rooms                       5462
expenses                   15164
properati_url                  0
barrio                         0
comuna                         0
dtype: int64

# Scikit-Learn, SimpleImputer
En general no queremos eliminar las instancias que posean NaNs en algun atributo / feature. Con imputer por ejemplo podemos reemplazar los campos con NaN por un valor determinado

In [7]:
# Imputer sirve para reemplazar los valores NaN con el valor de la mediana de dicho atributo
# Opciones de Aplicación: 'mean', 'median', 'most_frequent', 'constant'
imp = preprocessing.Imputer(missing_values='NaN', strategy='median', axis=0)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
# Función actual de Sklearn: SimpleImputer
# Opciones de Aplicación: 'mean', 'median', 'most_frequent', 'constant' 
imp = impute.SimpleImputer(missing_values= np.nan, strategy='median', fill_value=None) 

In [9]:
# Cuál es la mediana de esa columna?
properati['price_aprox_usd'].median()

180000.0

In [10]:
# aplicamos el valor de la mediana con la tecnica de Imputer en la columna "price_aprox_usd" y guardamos SOLO ESA COLUMNA
# en el elemento "properati_price_imp"
properati_price_imp = imp.fit_transform(properati[['price_aprox_usd']])

In [11]:
np.shape(properati[['price_aprox_usd']])

(18979, 1)

In [12]:
# luego revisamos si efectivamente sacamos todos los NaNs y es correcto
np.isnan(properati_price_imp).any()

False

In [13]:
# ahora reemplazamos nuestra columna con la nueva que tiene los valores reemplazados por la mediana
properati['price_aprox_usd'] = properati_price_imp

In [14]:
# nuevamente, observamos que esta columna ahora no tiene ningun valor nulo
properati['price_aprox_usd'].isnull().sum()

0

In [15]:
# por otro lado la mediana no ha sido afectada
properati['price_aprox_usd'].median()

180000.0

In [16]:
# Si tengo claro como quiero imputar, puedo hacerlo de varias columnas a la vez
properati_2 = imp.fit_transform(properati.iloc[:,7:14])

# Scikit-Learn: LabelEncoder
Existen casos donde si queremos transformar nuestras variables categoricas en numericas sin pasar por variables dummies, con LabelEncoder podemos realizar dicha tarea.

In [17]:
# Vemos los valores que toma la variable
properati.property_type.unique()

array(['house', 'store', 'apartment', 'PH'], dtype=object)

In [18]:
# Label Encoder transforma mis variables categoricas en numéricas.
le_proptype = preprocessing.LabelEncoder()

In [19]:
# "aprendimos" un array numerico con 4 valores posibles, uno por cada categoria
le_proptype.fit_transform(properati['property_type'])

array([2, 3, 3, ..., 3, 1, 1])

In [20]:
property_type_le = le_proptype.fit_transform(properati['property_type'])

In [21]:
le_proptype.classes_

array(['PH', 'apartment', 'house', 'store'], dtype=object)

In [22]:
# ahora la columna property_type del dataset properati tiene categorias numericas
properati['property_type_le'] = property_type_le

# Scikit-Learn: One Hot Encoder

In [23]:
# uso one hot encoder para transformar mis categorias numericas en categorias binarias
ohenc = preprocessing.OneHotEncoder(sparse = False)

In [24]:
np.shape(property_type_le)[0]

18979

In [25]:
# np.reshape sirve para cambiar las dimensiones de un array de numpy
property_type_le = np.reshape(property_type_le, (np.shape(property_type_le)[0],1))

In [26]:
np.shape(property_type_le)

(18979, 1)

In [27]:
property_type_le

array([[2],
       [3],
       [3],
       ...,
       [3],
       [1],
       [1]])

In [28]:
# con one hot encoder transformo mi vector de categorias (detalladas como numeros)
# en vectores de 1 y 0s , que tienen tantas posicoines como categorias.
onehot_encoded = ohenc.fit_transform(property_type_le)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [29]:
onehot_encoded

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [30]:
# transformo mi array de numpy "one hot encoded" a un dataframe y le asigno los nombres 
# de las columnas segun la categoria para acordarme que significa cada columna
onehot_pd = pd.DataFrame(onehot_encoded, index = properati.index , columns = le_proptype.classes_)

In [31]:
onehot_pd.head()

,PH,apartment,house,store
0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
